You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [8]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [9]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [10]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [11]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [12]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan for Blog Article on Artificial Intelligence

Outline:

I. Introduction
- Definition of Artificial Intelligence
- Importance of AI in various industries
- Brief overview of the latest trends in AI

II. Latest Trends in Artificial Intelligence
- Machine Learning advancements
- Natural Language Processing developments
- Robotics and Automation innovations
- AI in healthcare, finance, and other sectors

III. Key Players in the AI Industry
- Google
- IBM
- Amazon
- Microsoft
- Tesla
- Facebook

IV. N

I now can give a great answer

Final Answer:
# The Rise of Artificial Intelligence: Exploring Trends and Innovations

## Introduction

Artificial Intelligence (AI) has rapidly evolved in recent years, reshaping various industries and redefining the way businesses function. AI involves machines simulating human intelligence processes, particularly within computer systems. The impact of AI is profound, as it can analyze data, learn from patterns, and make decisions with minimal human intervention. This technology's potential for the future is immense, and understanding its current trends is crucial for staying ahead in a rapidly changing landscape.

## Latest Trends in Artificial Intelligence

### Machine Learning Advancements
Machine learning stands out as a key trend in AI, allowing computers to enhance their performance over time without explicit programming. From recommendation systems to autonomous vehicles, machine learning is revolutionizing various applications by learning from d

- Display the results of your execution as markdown in the notebook.

In [13]:
from IPython.display import Markdown
Markdown(result)

# The Rise of Artificial Intelligence: Exploring Trends and Innovations

## Introduction

Artificial Intelligence (AI) has rapidly evolved in recent years, reshaping various industries and redefining the way businesses function. AI involves machines simulating human intelligence processes, particularly within computer systems. The impact of AI is profound, as it can analyze data, learn from patterns, and make decisions with minimal human intervention. This technology's potential for the future is immense, and understanding its current trends is crucial for staying ahead in a rapidly changing landscape.

## Latest Trends in Artificial Intelligence

### Machine Learning Advancements
Machine learning stands out as a key trend in AI, allowing computers to enhance their performance over time without explicit programming. From recommendation systems to autonomous vehicles, machine learning is revolutionizing various applications by learning from data.

### Natural Language Processing Developments
The development of Natural Language Processing (NLP) has significantly improved communication between humans and machines. NLP focuses on computers' ability to understand, interpret, and generate human language, leading to advancements in chatbots, language translation, and sentiment analysis.

### Robotics and Automation Innovations
Robotics and automation are pivotal components of AI, transforming industries like manufacturing, logistics, and healthcare. Collaborative robots, or cobots, are enhancing efficiency and productivity by working alongside humans in various work environments.

### AI in Healthcare, Finance, and Other Sectors
AI's impact extends to sectors like healthcare and finance, where it aids in disease diagnosis, personalized treatment plans, fraud detection, and risk assessment. The integration of AI in these industries holds promise for improving patient outcomes, streamlining processes, and enhancing decision-making.

## Key Players in the AI Industry

Leading companies in the AI sector, including Google, IBM, Amazon, Microsoft, Tesla, and Facebook, are driving innovation and shaping the future of technology. These industry giants are constantly pushing boundaries, from self-driving cars to virtual assistants, showcasing the vast potential of AI.

## Noteworthy News on Artificial Intelligence

Recent breakthroughs in AI research have paved the way for practical applications in solving real-world problems, such as autonomous drones for disaster relief and AI-powered chatbots for customer service. However, ethical considerations in AI development are crucial to ensure responsible building and deployment of AI systems.

## Target Audience Analysis

Professionals in the technology sector, business owners seeking AI solutions, students and researchers interested in AI, and the general audience curious about AI's societal impact form the target audience for AI-related content. Staying informed about AI's latest developments provides valuable insights into the opportunities and challenges posed by this transformative technology.

## Conclusion

Navigating the evolving AI landscape requires staying informed about trends, innovations, and ethical considerations in the field. Understanding AI's impact on industries and society enables us to harness its full potential while addressing risks and challenges. Subscribe to our newsletter for regular updates on the world of Artificial Intelligence and join us in shaping the future of technology.

*Stay informed, stay ahead with Artificial Intelligence.*

*SEO Keywords: Artificial Intelligence trends 2021, Top AI companies, AI news updates, Impact of AI on industries, Ethical AI development*

Resources:
- [Forbes article on AI trends](https://www.forbes.com/ai-trends)
- [Gartner report on AI market](https://www.gartner.com/ai-market)
- [TechCrunch news on AI innovations](https://www.techcrunch.com/ai-innovations)
- [TED Talks on AI ethics](https://www.ted.com/ai-ethics)
- Research papers on AI advancements

Call to Action: Stay informed about the latest developments in Artificial Intelligence by subscribing to our newsletter for regular updates.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [ ]:
topic = "YOUR TOPIC HERE"
result = crew.kickoff(inputs={"topic": topic})

In [ ]:
Markdown(result)

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).